1. import data from ACLED
2. Connect to google sheets
3. paste to google sheets
4.

In [11]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [49]:
r = requests.get("http://acleddata.com/api/acled/read")

In [50]:
data = r.json()

In [51]:
data = data['data']

In [78]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('ACLED_to_Google_Sheets-e244e52c2f43.json', scope)

In [95]:
gc = gspread.authorize(credentials)

In [ ]:
sheet = gc.open('acled_test_2').sheet1

In [48]:
#for row in data:
#    sheet.append_row(row.values())

#### 1. Create a dataframe from data
2. Send columns to sheet
3. Send data to sheet

In [56]:
#create dataframe

df = pd.DataFrame(data)

In [75]:
def numberToLetters(q):
    q = q -1
    result = ''
    while q>= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

In [81]:
columns = df.columns.values.tolist()
cell_list = sheet.range('A1:'+numberToLetters(len(columns))+'1')
for cell in cell_list:
    val = columns[cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    cell.value = val
sheet.update_cells(cell_list)

In [88]:
len(df_acled_complete)

127229

In [89]:
127229+501

127730

In [73]:
num_lines, num_columns = df.shape

In [82]:
cell_list = sheet.range('A2:'+numberToLetters(num_columns)+str(num_lines+1))
for cell in cell_list:
    val = df.iloc[cell.row-2, cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    elif isinstance(val,(int, long, float, complex)):
        val= int(round(val))
    cell.value = val
sheet.update_cells(cell_list)

## the main sheet upload

In [83]:
df_acled_complete = pd.read_excel('acledmaster_tosql.xlsx', sheetname=0)

In [94]:
num_lines, num_columns = df_acled_complete.shape
cell_list = sheet.range('A1:Z127730')
for cell in cell_list:
    val = df_acled_complete.iloc[cell.row-2, cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    elif isinstance(val,(int, long, float, complex)):
        val= int(round(val))
    cell.value = val
sheet.update_cells(cell_list)

HTTPError: 401: <HTML>
<HEAD>
<TITLE>Unauthorized</TITLE>
</HEAD>
<BODY BGCOLOR="#FFFFFF" TEXT="#000000">
<H1>Unauthorized</H1>
<H2>Error 401</H2>
</BODY>
</HTML>


In [93]:
70000*26

1820000